In [50]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import numpy as np
from torch.nn.utils.rnn import pad_sequence

In [51]:
# Constants
CLEAN_DIR = 'C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\data\\clean_testset_wav'
NOISY_DIR = 'C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\data\\noisy_dataset_wav'

In [52]:
# Helper
def load_audio(file_path, sr=16000):
    audio, _ = librosa.load(file_path, sr=sr)
    return torch.tensor(audio).float().clone().detach()

In [53]:
# Dataset class
class NoiseReductionDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, sr=16000):
        self.noisy_files = sorted(os.listdir(noisy_dir))
        self.clean_files = sorted(os.listdir(clean_dir))
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.sr = sr

    def __len__(self):
        return len(self.noisy_files)

    def __getitem__(self, idx):
        noisy_path = os.path.join(self.noisy_dir, self.noisy_files[idx])
        clean_path = os.path.join(self.clean_dir, self.clean_files[idx])
        noisy = load_audio(noisy_path, self.sr)
        clean = load_audio(clean_path, self.sr)
        return noisy, clean

In [54]:
# Custom collate function to pad sequences in a batch
def collate_fn(batch):
    noisy_batch = [x[0] for x in batch]
    clean_batch = [x[1] for x in batch]
    
    # Padding sequences to the same length
    noisy_padded = pad_sequence(noisy_batch, batch_first=True, padding_value=0)
    clean_padded = pad_sequence(clean_batch, batch_first=True, padding_value=0)
    
    return noisy_padded, clean_padded

In [55]:
# Model class denoising CNN
class SimpleDenoisingCNN(nn.Module):
    def __init__(self):
        super(SimpleDenoisingCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=15, padding=7),
            nn.ReLU(),
            nn.Conv1d(16, 8, kernel_size=15, padding=7),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=15, padding=7),
            nn.ReLU(),
            nn.Conv1d(16, 1, kernel_size=15, padding=7)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [56]:
# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleDenoisingCNN()
model.to(device)

# Initialize optimizer and criterion
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dataset and DataLoader
dataset = NoiseReductionDataset(NOISY_DIR, CLEAN_DIR)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [57]:
# Training loop
num_epochs = 10
best_loss = float('inf')  # Initialize best loss to a large value

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for noisy, clean in dataloader:
        noisy, clean = noisy.to(device), clean.to(device)
        noisy = noisy.unsqueeze(1)  # Add channel dimension
        clean = clean.unsqueeze(1)  # Add channel dimension

        optimizer.zero_grad()
        output = model(noisy)
        loss = criterion(output, clean)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print loss after each epoch
    #print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

Epoch [1/10], Loss: 0.0006
Epoch [2/10], Loss: 0.0003
Epoch [3/10], Loss: 0.0003
Epoch [4/10], Loss: 0.0003
Epoch [5/10], Loss: 0.0003
Epoch [6/10], Loss: 0.0003
Epoch [7/10], Loss: 0.0003
Epoch [8/10], Loss: 0.0003
Epoch [9/10], Loss: 0.0003
Epoch [10/10], Loss: 0.0003


In [69]:
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [10/10], Loss: 0.0003


In [70]:
    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        print("💾 Saving the best model...")
        torch.save(model.state_dict(), 'denoising_model.pth')

    # Adjust learning rate
    scheduler.step()

💾 Saving the best model...


In [72]:
# Final model save
torch.save(model.state_dict(), 'denoising_model.pth')
print("💾 Final model saved to denoising_model.pth")

# Move the model to the correct directory
shutil.move("denoising_model.pth", "C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\model\\Denoising_model.pth")

💾 Final model saved to denoising_model.pth


'C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\model\\Denoising_model.pth'